# Problem 3: RBF network

Firstly, let's import the existing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from utils.data_parser import data_split_train_test, prepare_datasource
from utils.utilities import compute_cost, compute_accuracy #bring these utilities helpers from A1


Then, split out the datasource into training and test data with the ratio of 8:2

In [2]:
X, y = prepare_datasource()
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, shuffle=False)

Enter here


The spread parameters are preset as follow

In [3]:
radius_values = [0.05 ,0.1, 1., 2., 5., 10., 20., 30., 40., 50., 60., 70., 80., 90., 100., 120.]

Firstly, let's compute the gaussian matrix $$g_j=gaussian(d_j)=e^{\frac{-{d_j}^2}{2\sigma^2}}$$ where 
$d_j = ||X-c_j||$ and $c_j$ is each element in centers

In [4]:
def compute_gaussian_matrix(X, centers, width):
    gj = np.zeros((centers.shape[0], X.shape[0]))
    for c_idx,cj in enumerate(centers):
        for  input_idx,input in enumerate(X):
            dj = np.linalg.norm(input-cj)
            yj = np.exp(-dj**2/(2*(width**2)))
            gj[c_idx, input_idx] = yj
    return gj

Next, $g_j$ is fed into the linear regression model with proper weights to compute the output $$Y=\sum_{j=1}^n{w_jg_j}$$

In [5]:
def compute_output(X, weights, centers, width):
    G = compute_gaussian_matrix(X, centers, width)
    Y = np.matmul(weights, G)
    return Y

We now compute the weight matrix using gaussian matrix $G$ and the targets $Y$ by the formula:

$$W = G^{+}Y$$
where $G^{+}$ is the pseudo-inverse matrix of G

In [6]:
def compute_weight(X, Y, centers, width):
    G = compute_gaussian_matrix(X, centers, width)
    W = np.matmul(Y, np.linalg.pinv(G))
    return W

## Part 1
Carry out the design of RBF NN based on Gaussian kernel functions with constant spread function and using all
the points in the training set as centers of the RB functions. Compare the performance results (mean square
error) as you vary the spread parameter while keeping it the same for all kernel functions. Discuss your findings.

In [7]:
for radius in radius_values:
    W = compute_weight(X_train, y_train, X_train, radius)
    y_test_hat = compute_output(X_test, W, X_train, radius)
    y_test_pred = np.sign(y_test_hat)
    # y_train_hat = compute_output(X_train, W, X_train, radius)
    # y_train_pred = np.sign(y_train_hat)
    print(f"radius = {radius}   cost = {compute_cost(y_test_pred, y_test)} testing accuracy = {compute_accuracy(y_test_pred, y_test) * 100}")

radius = 0.05   cost = 0.1797752808988764 testing accuracy = 95.50561797752809
radius = 0.1   cost = 0.1797752808988764 testing accuracy = 95.50561797752809
radius = 1.0   cost = 0.2696629213483146 testing accuracy = 93.25842696629213
radius = 2.0   cost = 0.1797752808988764 testing accuracy = 95.50561797752809
radius = 5.0   cost = 0.1797752808988764 testing accuracy = 95.50561797752809
radius = 10.0   cost = 0.1797752808988764 testing accuracy = 95.50561797752809
radius = 20.0   cost = 0.0898876404494382 testing accuracy = 97.75280898876404
radius = 30.0   cost = 0.1348314606741573 testing accuracy = 96.62921348314607
radius = 40.0   cost = 0.1348314606741573 testing accuracy = 96.62921348314607
radius = 50.0   cost = 0.1797752808988764 testing accuracy = 95.50561797752809
radius = 60.0   cost = 0.1797752808988764 testing accuracy = 95.50561797752809
radius = 70.0   cost = 0.3146067415730337 testing accuracy = 92.13483146067416
radius = 80.0   cost = 0.7640449438202247 testing accura

## Part 2
Perform the design of the RBF NN, using this time only 150 centers, choosing the centers using two approaches:

a) Randomly select the centers from the input data.
b) Use K-Means algorithm to find the centers. You can use a Kmeans function defined in sklearn (https://scikitlearn.
org/stable/modules/generated/sklearn.cluster.KMeans.html) or create your own.

## 2.a. Randomly pick 150 centers 

In [8]:
selected_centers_indices = np.random.choice(X_train.shape[0], size=150, replace=False)
selected_centers = X_train[selected_centers_indices, :]


Then, run the whole thing again given the above centers:

In [9]:
for radius in radius_values:
    W = compute_weight(X_train, y_train, selected_centers, radius)
    y_hat = compute_output(X_test, W, selected_centers, radius)
    y_pred = np.sign(y_hat)
    print(f"radius = {radius}   cost = {compute_cost(y_pred, y_test)}   testing accuracy = {compute_accuracy(y_pred, y_test) * 100}")

radius = 0.05   cost = 0.1348314606741573   accuracy = 96.62921348314607
radius = 0.1   cost = 0.1348314606741573   accuracy = 96.62921348314607
radius = 1.0   cost = 0.1348314606741573   accuracy = 96.62921348314607
radius = 2.0   cost = 0.1797752808988764   accuracy = 95.50561797752809
radius = 5.0   cost = 0.1797752808988764   accuracy = 95.50561797752809
radius = 10.0   cost = 0.1797752808988764   accuracy = 95.50561797752809
radius = 20.0   cost = 0.1348314606741573   accuracy = 96.62921348314607
radius = 30.0   cost = 0.1348314606741573   accuracy = 96.62921348314607
radius = 40.0   cost = 0.1348314606741573   accuracy = 96.62921348314607
radius = 50.0   cost = 0.0898876404494382   accuracy = 97.75280898876404
radius = 60.0   cost = 0.1797752808988764   accuracy = 95.50561797752809
radius = 70.0   cost = 0.3146067415730337   accuracy = 92.13483146067416
radius = 80.0   cost = 0.7640449438202247   accuracy = 80.89887640449437
radius = 90.0   cost = 0.7640449438202247   accuracy = 

## 2.b. Use K-Means algorithm for selecting 150 centers 

In [10]:
kmeans = KMeans(n_clusters=150)
kmeans.fit(X_train)
kmean_centers = kmeans.cluster_centers_

/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Finally, execute the program with the centers given by KMeans clusters

In [11]:
for radius in radius_values:
    W = compute_weight(X_train, y_train, kmean_centers, radius)
    y_hat = compute_output(X_test, W, kmean_centers, radius)
    y_pred = np.sign(y_hat)
    print(f"radius = {radius}   cost = {compute_cost(y_pred, y_test)}   testing accuracy = {compute_accuracy(y_pred, y_test) * 100}")

radius = 0.05   cost = 0.1797752808988764   accuracy = 95.50561797752809
radius = 0.1   cost = 0.1797752808988764   accuracy = 95.50561797752809
radius = 1.0   cost = 0.1348314606741573   accuracy = 96.62921348314607
radius = 2.0   cost = 0.1797752808988764   accuracy = 95.50561797752809
radius = 5.0   cost = 0.1797752808988764   accuracy = 95.50561797752809
radius = 10.0   cost = 0.1797752808988764   accuracy = 95.50561797752809
radius = 20.0   cost = 0.1348314606741573   accuracy = 96.62921348314607
radius = 30.0   cost = 0.1348314606741573   accuracy = 96.62921348314607
radius = 40.0   cost = 0.1348314606741573   accuracy = 96.62921348314607
radius = 50.0   cost = 0.1348314606741573   accuracy = 96.62921348314607
radius = 60.0   cost = 0.1797752808988764   accuracy = 95.50561797752809
radius = 70.0   cost = 0.1797752808988764   accuracy = 95.50561797752809
radius = 80.0   cost = 0.7640449438202247   accuracy = 80.89887640449437
radius = 90.0   cost = 0.7640449438202247   accuracy = 

## Analysis